<a href="https://colab.research.google.com/github/apeetdkl/Python-DS/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np

# Step 1: Load all three data files
csv_data = pd.read_csv('covid_data.csv')
excel_data = pd.read_excel('covid_data.xlsx', engine='openpyxl')
json_data = pd.read_json('covid_data.json')

# Step 2: Preprocess CSV data
csv_data['Date'] = pd.to_datetime(csv_data['Date'], errors='coerce')
csv_data.rename(columns={'Name of State / UT': 'Location'}, inplace=True)
csv_data['Source'] = 'India_CSV'

# Step 3: Preprocess Excel data
excel_data['Date'] = pd.to_datetime(excel_data[['year', 'month', 'day']], errors='coerce')
excel_data.rename(columns={'countriesAndTerritories': 'Location'}, inplace=True)
excel_data['Source'] = 'Global_Excel'

# Step 4: Preprocess JSON data
# Try to use 'date' or create a placeholder if no date
if 'date' in json_data.columns:
    json_data['Date'] = pd.to_datetime(json_data['date'], errors='coerce')
else:
    json_data['Date'] = pd.NaT  # missing dates
json_data['Location'] = json_data.get('location', 'Unknown')  # fallback if no location
json_data['Source'] = 'JSON'

# Step 5: Add mock 'Test_Result' data
test_results_pool = ['Positive', 'Negative', 'POS', 'Covid+ve', 'Neg.', 'Covid-ve']
for df in [csv_data, excel_data, json_data]:
    df['Test_Result'] = np.random.choice(test_results_pool, size=len(df))

# Step 6: Normalize test result labels
def clean_test_result(result):
    result = str(result).lower()
    if 'pos' in result or '+ve' in result:
        return 'Positive'
    elif 'neg' in result or '-ve' in result:
        return 'Negative'
    return None

for df in [csv_data, excel_data, json_data]:
    df['Test_Result'] = df['Test_Result'].apply(clean_test_result)

# Step 7: Combine datasets
combined_data = pd.concat([csv_data, excel_data, json_data], ignore_index=True)

# Step 8: Drop duplicates by Location, keeping the latest by Date
combined_data.sort_values(by='Date', ascending=False, inplace=True)
combined_data.drop_duplicates(subset=['Location'], keep='first', inplace=True)

# Step 9: Infer vaccination status (mock: visited center = vaccinated)
combined_data['Visited_Vaccine_Center'] = np.random.choice([True, False], size=len(combined_data))
combined_data['Vaccinated'] = combined_data['Visited_Vaccine_Center'].apply(lambda x: 'Yes' if x else 'No')

# Step 10: Final cleaned dataset
final_columns = ['Date', 'Location', 'Test_Result', 'Vaccinated', 'Source']
final_data = combined_data[final_columns]

# Step 11: Output preview
print(final_data.head())

# Optional: Save to CSV
final_data.to_csv('cleaned_covid_data.csv', index=False)

            Date           Location Test_Result Vaccinated        Source
50917 2020-12-14              Qatar    Negative        Yes  Global_Excel
29444 2020-12-14      Guinea_Bissau    Negative        Yes  Global_Excel
13296 2020-12-14  Brunei_Darussalam    Positive        Yes  Global_Excel
56812 2020-12-14        South_Korea    Negative        Yes  Global_Excel
9290  2020-12-14         Bangladesh    Negative         No  Global_Excel


In [1]:
pip install pandas openpyxl


In [16]:
import pandas as pd
import numpy as np

# Step 1: Load the hospital data
hospital_data = pd.read_csv('hospital_covid_data.csv')

# Step 2: Fix column typo if needed
hospital_data.rename(columns=lambda x: x.strip(), inplace=True)
if 'ocation' in hospital_data.columns:
    hospital_data.rename(columns={'ocation': 'Location'}, inplace=True)

# Step 3: Add mock columns to demonstrate cleaning steps
np.random.seed(42)  # for reproducibility
hospital_data['Name'] = np.random.choice(['joHN doE', 'Alice SMITH', 'Dr. Michael Brown', 'sARAh kHan'], size=len(hospital_data))
hospital_data['SpO2'] = np.random.randint(35, 105, size=len(hospital_data))  # some invalid values
hospital_data['Age'] = np.random.randint(-20, 210, size=len(hospital_data))  # some unrealistic ages
hospital_data['Hospitalized'] = np.nan  # initially missing

# Step 4: Standardize patient names (title case, remove "Dr.")
def clean_name(name):
    name = str(name).replace("Dr.", "").strip()
    return name.title()

hospital_data['Name'] = hospital_data['Name'].apply(clean_name)

# Step 5: Fix invalid SpO2 values (<40 or >100)
hospital_data['SpO2'] = hospital_data['SpO2'].apply(lambda x: np.nan if x < 40 or x > 100 else x)

# Step 6: Fix unrealistic ages (<0 or >120)
hospital_data['Age'] = hospital_data['Age'].apply(lambda x: np.nan if x < 0 or x > 120 else x)

# Step 7: Infer hospitalization if SpO2 < 90%
hospital_data['Hospitalized'] = hospital_data.apply(
    lambda row: 'Yes' if pd.notna(row['SpO2']) and row['SpO2'] < 90 else 'No', axis=1
)

# Step 8: Final cleaned dataset (without 'Location')
final_columns = ['Name', 'Age', 'SpO2', 'Hospitalized']
final_cleaned_data = hospital_data[final_columns]

# Step 9: Preview cleaned data
print(final_cleaned_data.head())

# Optional: Save cleaned data
final_cleaned_data.to_csv('cleaned_hospital_covid_data.csv', index=False)

            Name    Age  SpO2 Hospitalized
0  Michael Brown    NaN  96.0           No
1     Sarah Khan   42.0  85.0          Yes
2       John Doe  118.0  89.0          Yes
3  Michael Brown   60.0  98.0           No
4  Michael Brown  115.0   NaN           No
